In [1]:
import os

In [2]:
pwd


'c:\\Users\\Robin Aluma\\Desktop\\Car_Price_Pred\\Resources'

In [3]:
os.chdir('../')

In [4]:
pwd


'c:\\Users\\Robin Aluma\\Desktop\\Car_Price_Pred'

In [5]:

from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np
import sys
import pandas as pd

In [6]:

from src.Car_Price_Pred.utils.common import load_models,load_parameters,read_yaml,create_directories,save_best_model
from dataclasses import dataclass
from pathlib import Path
from src.Car_Price_Pred.constants import *
from sklearn.model_selection import train_test_split
from Exceptions import CustomException

In [7]:
@dataclass
class ModelEvaluationConfig():
    root_url: Path
    evaluation_data: Path
    best_model_path: Path
    decision_tree: str
    lasso: str
    linear_regression: str
    neighbors: str
    svm: str
    random_forest: str
    

In [8]:
class ModelEvaluationConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH):
        self.config = read_yaml(config_file_path)
        
        create_directories([self.config.Artifacts_root])
        
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_url])
        model_evaluation_config = ModelEvaluationConfig(
            root_url = config.root_url,
            evaluation_data = config.evaluation_data,
            best_model_path = config.best_model_path,
            decision_tree = config.decision_tree,
            lasso = config.lasso,
            linear_regression = config.linear_regression,
            neighbors = config.neighbors,
            svm = config.svm,
            random_forest = config.random_forest
        )
        
        return model_evaluation_config
    

In [9]:
class ModelEvaluation():
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config
        
    def evaluate_model(self):
        test_data = pd.read_csv(self.config.evaluation_data)
        
        test_data.fillna(test_data.mean(),inplace=True)
        
        self.x_test = test_data.iloc[:,:-1]
        self.y_test = test_data.iloc[:,-1]
        x_train,x_test,y_train,y_test = train_test_split(self.x_test,self.y_test,test_size=0.3,random_state=42)
        
        models = [
            load_models(self.config.decision_tree),
            load_models(self.config.lasso),
            load_models(self.config.linear_regression),
            load_models(self.config.neighbors),
            load_models(self.config.svm),
            load_models(self.config.random_forest)]
        
        best_model = None
        best_score = 0
        
        for model in models:
            model.fit(x_train,y_train)
            self.y_predict = model.predict(x_test)
            score = r2_score(y_test,self.y_predict)
            if score > best_score:
                best_model = model
                best_score = score
        
        save_best_model(best_model,os.path.join(self.config.best_model_path,f"{best_model}_.joblib"))
        

In [ ]:
class ModelEvaluationPipeline():
    def __init__(self):
        pass
    def main(self):
        try:
            config = ModelEvaluationConfigurationManager()
            get_model_evaluation_config = config.get_model_evaluation_config()
            model_evaluation = ModelEvaluation(config=get_model_evaluation_config)
            model_evaluation.evaluate_model()
        except Exception as e:
            raise CustomException(e,sys)

In [11]:
if __name__ == '__main__':
    try:
        obj = ModelEvaluationPipeline()
        obj.main()
    except Exception as e:
        raise CustomException(e,sys)

[ 2025-01-22 12:43:34,901 : common : INFO : Creates directory for path;artifacts in file paths :['artifacts'] ]
[ 2025-01-22 12:43:34,903 : common : INFO : Creates directory for path;artifacts/model_evaluation in file paths :['artifacts/model_evaluation'] ]
